In [129]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import re

from sklearn.preprocessing import scale
from sklearn import linear_model
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

In [130]:
score = pd.read_csv("score_final.csv", encoding="cp949")

In [131]:
def preprocessiong(score):
    # 참여인원 (int) 숫자만 남기기
    score["참여인원"] = [k.replace(",","") for k in score["참여인원"]]
    score["참여인원"] = [k.replace("명","") for k in score["참여인원"]]
    score["참여인원"] = [k.replace("\r","") for k in score["참여인원"]]
    score["참여인원"] = [k.replace("\n","") for k in score["참여인원"]]
    score["참여인원"] = [int(k) for k in score["참여인원"]]
    
    # 글쎄요/보고싶어요 0인 데이터 지우기
    score = score[score["글쎄요"] != 0]
    score = score[score["보고싶어요"] != 0]
    
    score["관심인원"] = score["글쎄요"] + score["보고싶어요"]
    score["기대지수"] = score["보고싶어요"] / score["관심인원"]
    
    # id 한글만 남기고 / id 길이 변수 만들기
    score["id"] = [re.sub("[2, , :, ,]","", id) for id in score["id"]]
    score["id_length"] = [len(i) for i in score["id"]]
    
    # 런타임 (int) 숫자만 남기기
    score["런타임"] = [k.replace("분","") for k in score["런타임"]]
    score["런타임"] = [int(k) for k in score["런타임"]]
    
    # 등급 숫자만 남기기
    score["등급"] = [k.replace("관람가","") for k in score["등급"]]
    score["등급"] = [k.replace(" ","") for k in score["등급"]]
    
    # 개봉월 / 개봉년도 나누기
    score["개봉년도"] = [k.split(".")[0] for k in score["개봉일"]]
    score["개봉년도"] = [int(k) for k in score["개봉년도"]]
    score["개봉월"] = [k.split(".")[1] for k in score["개봉일"]]
    score["개봉월"] = [int(k) for k in score["개봉월"]]

    # 주연 수 만들기
    score["주연수"] = [len(k.split(",")) for k in score["주연"]]
    score["조연수"] = [len(k.split(",")) for k in score["조연"]]
    score["배우수"] = score["주연수"] + score["조연수"]


    # 장르 string 필요없는 것 제거
    k = [i.replace("[","") for i in score["장르"]]
    k = [i.replace("]","") for i in k]
    k = [i.replace("'","") for i in k]
    k = [i.replace(" ","") for i in k]
    k = [i.split(",") for i in k] 
    score["장르"] = k
    genre_unique = np.unique([b for a in score["장르"] for b in a])
    result = []
    for b in score["장르"]:
        d=[]
        for a in genre_unique:
            d.append(int(a in b))
        result.append(d)
    genre_dummy = pd.DataFrame(result, columns=genre_unique)
    return score, genre_dummy

In [132]:
score, dummy = preprocessiong(score)
score.index = [i for i in range(192)]
score = score.join(dummy)

In [133]:
def delete(score):
    del score["Unnamed: 0"]
    del score["장르"]
    del score["등급"]
    del score["개봉일"]
    del score["주연"]
    del score["조연"]
    del score["id"]
    del score["감독"]
delete(score)

In [134]:
y = score["y"]
score = score.drop("y", axis=1)

In [135]:
from sklearn.cross_validation import train_test_split

train_X, test_X = train_test_split(score, test_size = 0.3, random_state=1) # 독립변수만
train_Y, test_Y = train_test_split(y, test_size = 0.3, random_state=1) # 종속변수만|

In [126]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier

model1 = LogisticRegression(random_state=1)
model2 = QuadraticDiscriminantAnalysis()
model3 = GaussianNB()
ensemble = VotingClassifier(estimators=[('lr', model1), ('qda', model2), ('gnb', model3)], 
                            voting='soft', weights=[1, 1, 2])

In [137]:
pred_en = ensemble.fit(train_X, train_Y).predict(test_X)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
